In [1]:
import numpy as np
import time

%load_ext autoreload
%aimport convolve
%autoreload 1

# Small manual testing

In [2]:
dim_input = 1000
dim_W = 9
# input = np.arange(dim_input ** 2, dtype=float).reshape((dim_input, dim_input))
# W = np.arange(dim_W ** 2, dtype=float).reshape((dim_W, dim_W))

dim_input = 3
dim_W = 2
input = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=float)
W = np.array([[1, 2], [3, 4]], dtype=float)
padding = 1
stride = 1
dim_out = (dim_input - W.shape[0] + 2 * padding) // stride + 1
output = np.zeros((dim_out, dim_out), dtype=float)
input = input.reshape((dim_input * dim_input,))
W = W.reshape((W.shape[0] * W.shape[1],))
output = output.reshape((dim_out ** 2,))
start = time.clock()
convolve.conv_forward(input, W, dim_input, dim_W, padding, stride, output)
dJ_dout = np.array([[1, 3, 2, 4], [0, 5, 1, 6], [2, 7, 9, 8], [0, 3, 1, 5]], dtype=float)
# dJ_dout = np.array([[1, 3], [0, 5]], dtype=float)
dJ_dout = dJ_dout.reshape((dim_out * dim_out,))
dJ_dW = np.zeros((dim_W * dim_W,), dtype=float)
dJ_din = np.zeros((dim_input * dim_input,), dtype=float)
convolve.conv_backward_W(dJ_dout, input, dim_input, dim_W, padding, stride, dJ_dW)
convolve.conv_backward_input(dJ_dout, W, dim_input, dim_W, padding, stride, dJ_din)
print(time.clock() - start)
print('input\n', input.reshape((dim_input, dim_input)))
print('W\n', W.reshape((dim_W, dim_W)))
print('output\n', output.reshape((dim_out, dim_out)))
print('dJ_dout\n', dJ_dout.reshape((dim_out, dim_out)))
# print('dJ_dW\n', dJ_dW.reshape((dim_W, dim_W)))
print('dJ_din\n', dJ_din.reshape((dim_input, dim_input)))

0.0007490000000001107
input
 [[ 1.  2.  3.]
 [ 4.  5.  6.]
 [ 7.  8.  9.]]
W
 [[ 1.  2.]
 [ 3.  4.]]
output
 [[  4.  11.  18.   9.]
 [ 18.  37.  47.  21.]
 [ 36.  67.  77.  33.]
 [ 14.  23.  26.   9.]]
dJ_dout
 [[ 1.  3.  2.  4.]
 [ 0.  5.  1.  6.]
 [ 2.  7.  9.  8.]
 [ 0.  3.  1.  5.]]
dJ_din
 [[ 18.  29.  28.]
 [ 26.  46.  48.]
 [ 32.  62.  67.]]


# Large testing using small delta derivative approximation

In [33]:
dim_input = 100
dim_W = 9
input = np.arange(dim_input ** 2, dtype=float).reshape((dim_input, dim_input))
W = np.arange(dim_W ** 2, dtype=float).reshape((dim_W, dim_W))

start = time.clock()
padding = 4
stride = 6
dim_out = (dim_input - dim_W + 2 * padding) // stride + 1
output = np.zeros((dim_out * dim_out,), dtype=float)
input = input.reshape((dim_input * dim_input,))
W = W.reshape((dim_W * dim_W,))

convolve.conv_forward(input, W, dim_input, dim_W, padding, stride, output)
V = np.random.random((dim_out * dim_out,))
J = np.dot(output, V)

print('J:', J)

dJ_dout = V

dJ_dW = np.zeros((dim_W * dim_W,), dtype=float)
dJ_din = np.zeros((dim_input * dim_input,), dtype=float)
convolve.conv_backward_W(dJ_dout, input, dim_input, dim_W, padding, stride, dJ_dW)
convolve.conv_backward_input(dJ_dout, W, dim_input, dim_W, padding, stride, dJ_din)

print(time.clock() - start)

out_changed = np.zeros((dim_out * dim_out,), dtype=float)
dJ_dW_approx = np.zeros((dim_W * dim_W,), dtype=float)
delta = 1e-5
for i, w_original in enumerate(W):
    W[i] = w_original + delta
    convolve.conv_forward(input, W, dim_input, dim_W, padding, stride, out_changed)
    J_changed = np.dot(out_changed, V)
    dJ_dW_approx[i] = (J_changed - J) / delta
    W[i] = w_original

print('dJ_dW magnitude', np.sum(np.abs(dJ_dW)))
print('total error from approximation', np.sum(np.abs(dJ_dW - dJ_dW_approx)))
    
dJ_din_approx = np.zeros((dim_input * dim_input,), dtype=float)
for i, x in enumerate(input):
    input[i] = x + delta
    convolve.conv_forward(input, W, dim_input, dim_W, padding, stride, out_changed)
    J_changed = np.dot(out_changed, V)
    dJ_din_approx[i] = (J_changed - J) / delta
    input[i] = x
    
print('dJ_din magnitude', np.sum(np.abs(dJ_din)))
print('total error from approximation', np.sum(np.abs(dJ_din - dJ_din_approx)))

J: 2264941352.89
0.0016200000000026193
dJ_dW magnitude 55584819.6805
total error from approximation 2.84990129864
dJ_din magnitude 466800.631859
total error from approximation 365.607067495
